In [ ]:
import numpy as np
from matplotlib.pylab import *
from numpy import random
from scipy.linalg import hadamard
import pdb

In [ ]:
def figsize(width,height):
    rcParams['figure.figsize'] = (width,height)

In [ ]:
#%matplotlib notebook
%matplotlib inline

figsize(15, 8)

In [ ]:
#plot(rand(10))

## Functions

In [ ]:
def _B_pm1(d, rng=None):
    if rng is None:
        return random.randint(2, size=(d)) * 2 - 1
    else:
        return rng.randint(2, size=(d)) * 2 - 1

def B_pm1(d, rng=None):
    return diag(_B_pm1(d, rng=rng))

def _G_gauss(d, rng=None):
    if rng is None:
        return random.normal(0, 1, d)
    else:
        return rng.normal(0, 1, d)

def G_gauss(d, rng=None):
    return diag(_G_gauss(d, rng=rng))

def _Pi_perm(x, rng=None):
    '''Fast perm, apply right away'''
    d = x.shape[0]
    if rng is None:
        return x[random.permutation(d)]
    else:
        return x[rng.permutation(d)]
    
def _Pi_perm_order(d, rng=None):
    '''Fast perm, return perm order'''
    if rng is None:
        return random.permutation(d)
    else:
        return rng.permutation(d)
    
def Pi_perm(d, rng=None):
    '''Slow perm matrix'''
    if rng is None:
        return eye(d)[random.permutation(d)]
    else:
        return eye(d)[rng.permutation(d)]

## Dense version

In [ ]:
d = 2**9
n = 10*d
random.seed(1246)

In [ ]:
proj = random.normal(0, 1, (n, d))
#proj *= 1. / sqrt((proj**2).sum(1)).reshape(n, 1)
#proj /= sqrt((proj**2).sum())
proj /= sqrt((proj**2).sum(0))[newaxis,:]

In [ ]:
(proj**2).sum(0)[:10]

In [ ]:
x = random.normal(0, 1, (d, 1))

In [ ]:
x[:10]

In [ ]:
y = dot(proj, x)

In [ ]:
y[:10]

Check distances

In [ ]:
delta = random.normal(0, 1, (d, 1))

In [ ]:
y_delta = dot(proj, x) - dot(proj, x + delta)

In [ ]:
(delta**2).sum()

In [ ]:
(y_delta**2).sum()

In [ ]:
%%time
proj = random.normal(0, 1, (n, d))
proj /= sqrt((proj**2).sum(0))[newaxis,:]

In [ ]:
%%time
y = dot(proj, x)

In [ ]:
sys.getsizeof(proj) / 1e6

In [ ]:
del proj

In [ ]:
_=hist(delta, bins=30)

In [ ]:
_=hist(y_delta, bins=30)

## Manual version

In [ ]:
d = 8
n = 24

In [ ]:
B = B_pm1(d)
H = hadamard(d)
Pi = Pi_perm(d)
G = G_gauss(d)

In [ ]:
proj = dot(H, dot(G, dot(Pi, dot(H, B))))

## Slow version

In [ ]:
class ProjectSlow(object):
    def __init__(self, d, n, seed=123):
        self.d = d
        self.n = n
        self.rng = random.RandomState(seed)
        
        self.B = []
        self.H = hadamard(self.d)
        self.Pi = []
        self.G = []

        self.float_replicates = float(self.n)/self.d
        self.replicates = int(ceil(self.float_replicates))
        
        for ii in range(self.replicates):
            self.B.append(B_pm1(d, rng=self.rng))
            self.Pi.append(Pi_perm(d, rng=self.rng))
            self.G.append(G_gauss(d, rng=self.rng))

    def project_i(self, x, i):
        norm_by = sqrt((self.G[i]**2).sum() * self.d)
        ret = dot(self.H, dot(self.G[i], dot(self.Pi[i], dot(self.H, dot(self.B[i], x)))))
        ret /= norm_by
        ret /= sqrt(self.float_replicates)
        return ret
    
    def project(self, x):
        rets = []
        for ii in range(self.replicates):
            rets.append(self.project_i(x, ii))
        return vstack(rets)
            
            
            


In [ ]:
d = 2**9
n = 10*d

In [ ]:
pp = ProjectSlow(d, n, seed=123)

In [ ]:
random.seed(1234)

In [ ]:
x = random.normal(0, 1, (d, 1))

In [ ]:
x

In [ ]:
y = pp.project(x)

Check distances

In [ ]:
delta = random.normal(0, 1, (d, 1))

In [ ]:
y_delta = pp.project(x) - pp.project(x + delta)

In [ ]:
(delta**2).sum()

In [ ]:
(y_delta**2).sum()

In [ ]:
%%time
pp = ProjectSlow(d, n, seed=123)

In [ ]:
%%time
pp.project(x)

## Fast version?

In [ ]:
class ProjectFast(object):
    def __init__(self, d, n, seed=123):
        self.d = d
        self.n = n
        self.rng = random.RandomState(seed)
        
        self.B = []
        self.H = hadamard(self.d)
        self.Pi = []
        self.G = []

        self.float_replicates = float(self.n)/self.d
        self.replicates = int(ceil(self.float_replicates))
        
        for ii in range(self.replicates):
            self.B.append(_B_pm1(d, rng=self.rng)[:,newaxis])
            self.Pi.append(_Pi_perm_order(d, rng=self.rng))
            self.G.append(_G_gauss(d, rng=self.rng)[:,newaxis])

    def project_i(self, x, i):
        #print i
        #pdb.set_trace()
        norm_by = sqrt((self.G[i]**2).sum() * self.d)
        #norm_by = sqrt((self.G[i]**2).sum())
        ret = self.B[i] * x
        ret = dot(self.H, ret)
        ret = ret[self.Pi[i]]
        ret = self.G[i] * ret
        ret = dot(self.H, ret)
        ret /= norm_by
        ret /= sqrt(self.float_replicates)
        return ret
    
    def project(self, x):
        rets = []
        for ii in range(self.replicates):
            rets.append(self.project_i(x, ii))
        return vstack(rets)
            
            
            


In [ ]:
d = 2**13
n = 10*d
random.seed(1234)
x = random.normal(0, 1, (d, 1))
delta = random.normal(0, 1, (d, 1))

In [ ]:
pp = ProjectFast(d, n, seed=123)
#pp = ProjectSlow(d, n, seed=123)

Check distances

In [ ]:
y_delta = pp.project(x) - pp.project(x + delta)

In [ ]:
(delta**2).sum()

In [ ]:
(y_delta**2).sum()

In [ ]:
%%time
pp = ProjectFast(d, n, seed=123)

In [ ]:
%%time
pp.project(x)

In [ ]:
sys.getsizeof(pp.H) / 1e6

# Implement Fast Walsh-Hadamard transform

In [ ]:
def fwht(x):
    # x is [d,1] where d is a power of 2
    if x.shape[0] == 1:
        return x
    else:
        x_top = x[:int(x.shape[0]/2)]
        x_bot = x[int(x.shape[0]/2):]
        return vstack([fwht(x_top+x_bot), fwht(x_top-x_bot)])

In [ ]:
a = np.arange(8)
print(a[0::2])
print(a[1::2])
print(a[0::2] + a[1::2])
print(a[0::2] - a[1::2])

In [ ]:
def fwht_opt(x):
    """
    Modified version of: https://github.com/dingluo/fwht/blob/master/FWHT.py
    Fast Walsh-Hadamard Transform
    Based on mex function written by Chengbo Li@Rice Uni for his TVAL3 algorithm.
    His code is according to the K.G. Beauchamp's book -- Applications of Walsh and Related Functions.
    """
    x = x.squeeze()
    N = x.size
    G = int(N/2) # Number of Groups
    M = 2 # Number of Members in Each Group

    # First stage
    y = np.zeros((int(N/2),2))
    y[:,0] = x[0::2] + x[1::2]
    y[:,1] = x[0::2] - x[1::2]
    x = y.copy()
    # Second and further stage
    for nStage in range(2,int(math.log(N,2))+1):
        y = np.zeros((int(G/2),M*2))
        y[0:int(G/2),0:M*2:4] = x[0:G:2,0:M:2] + x[1:G:2,0:M:2]
        y[0:int(G/2),1:M*2:4] = x[0:G:2,0:M:2] - x[1:G:2,0:M:2]
        y[0:int(G/2),2:M*2:4] = x[0:G:2,1:M:2] - x[1:G:2,1:M:2]
        y[0:int(G/2),3:M*2:4] = x[0:G:2,1:M:2] + x[1:G:2,1:M:2]
        x = y.copy()
        G = int(G/2)
        M = M*2
    x = y[0,:]
    x = x.reshape((x.size,1))
    return x

In [ ]:
class ProjectFaster(object):
    def __init__(self, d, n, seed=123):
        self.d = d
        self.n = n
        self.rng = random.RandomState(seed)
        
        self.B = []
        self.Pi = []
        self.G = []

        self.float_replicates = float(self.n)/self.d
        self.replicates = int(ceil(self.float_replicates))
        
        for ii in range(self.replicates):
            self.B.append(_B_pm1(d, rng=self.rng)[:,newaxis])
            self.Pi.append(_Pi_perm_order(d, rng=self.rng))
            self.G.append(_G_gauss(d, rng=self.rng)[:,newaxis])

    def project_i(self, x, i):
        #print i
        #pdb.set_trace()
        norm_by = sqrt((self.G[i]**2).sum() * self.d)
        #norm_by = sqrt((self.G[i]**2).sum())
        ret = self.B[i] * x
        ret = fwht_opt(ret)
        ret = ret[self.Pi[i]]
        ret = self.G[i] * ret
        ret = fwht_opt(ret)
        ret /= norm_by
        ret /= sqrt(self.float_replicates)
        return ret
    
    def project(self, x):
        rets = []
        for ii in range(self.replicates):
            rets.append(self.project_i(x, ii))
        return vstack(rets)


In [ ]:
d = 2**13
n = 10*d
random.seed(1234)
x = random.normal(0, 1, (d, 1))
delta = random.normal(0, 1, (d, 1))

In [ ]:
pp = ProjectFaster(d, n, seed=123)

In [ ]:
y_delta = pp.project(x) - pp.project(x + delta)

In [ ]:
(delta**2).sum()

In [ ]:
(y_delta**2).sum()

In [ ]:
%%time
pp = ProjectFaster(d, n, seed=123)

In [ ]:
%%time
pp.project(x)

# Just Hadamard naive vs FFT

In [ ]:
d = 8
n = 8
random.seed(1234)
x = 1.0 * arange(d) ** 2

In [ ]:
x

In [ ]:
H = hadamard(d)

In [ ]:
H

In [ ]:
hx = dot(H, x)
hx

In [ ]:
plot(x, 'ko-')
plot(hx, 'bo-', ms=10)
#plot(hx, 'ro-')

In [ ]:
fft(x)

In [ ]:
absolute(fft(x))

In [ ]:
x

In [ ]:
def numpy_hadamard(x):
    '''Hadamard transform along last dim of x.'''
    orig_shape = x.shape
    assert len(orig_shape) in (1, 2), 'x should be a vector or matrix'
    if len(x.shape) == 1:
        x = x.reshape((1,) + x.shape)
        
    nn = x.shape[-1]
    ll = int(round(log(nn) / log(2)))
    assert 2**ll == nn, 'x shape last dim must be a power of 2'
    expanded_shape = x.shape[0] + tuple([2] * ll)
    ret = x.reshape(expanded_shape)

    for ii in range(ll):
        print ii
        ret = 
    ret = ret.reshape(orig_shape)
    return


In [ ]:
numpy_hadamard(x)

# Reshape-based Hadamard implementations

## numpy

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from fast_walsh_hadamard import np_fast_walsh_hadamard

#### Wikipedia example: https://en.wikipedia.org/wiki/Hadamard_transform

In [ ]:
x = np.array([1,0,1,0,0,1,1,0])

In [ ]:
H = hadamard(len(x))

In [ ]:
dot(x, H)

In [ ]:
np_fast_walsh_hadamard(x, axis=0, normalize=False)

In [ ]:
fwht(x).flatten()

In [ ]:
print 'This one is different'
fwht_opt(x).flatten()

But applying it twice results in the original vector (rescaled), so at least it's consistent

In [ ]:
x

In [ ]:
fwht_opt(fwht_opt(x).flatten()).flatten() / 8

#### Large random matrix

In [ ]:
d = 2**10
n = 1000
random.seed(1234)
#x = 1.0 * arange(d) ** 2
x = random.normal(0, 1, (n, d))

In [ ]:
ret = np_fast_walsh_hadamard(x, axis=1)
retret = np_fast_walsh_hadamard(ret, axis=1)

In [ ]:
((retret - x)**2).sum()

In [ ]:
%%time
_ = np_fast_walsh_hadamard(x, axis=1)

## Tensorflow, Hadamard Transform

In [ ]:
import tensorflow as tf
from fast_walsh_hadamard import tf_fast_walsh_hadamard

In [ ]:
in_x = tf.placeholder('float32', name='in_x')

In [ ]:
hx = tf_fast_walsh_hadamard(in_x, axis=0, normalize=False)

In [ ]:
sess = tf.InteractiveSession()

#### Wikipedia example: https://en.wikipedia.org/wiki/Hadamard_transform

In [ ]:
x = np.array([1,0,1,0,0,1,1,0])

In [ ]:
H = hadamard(len(x))

In [ ]:
dot(x, H)

In [ ]:
%%time
sess.run(hx, feed_dict={in_x: x})

## Check of Fastfood transform properties

In [ ]:
D = 512
random.seed(1235)
#x = 1.0 * arange(d) ** 2
#x = random.normal(0, 1, (n, d))

In [ ]:
B = diag(random.choice((-1, 1), D))
H = hadamard(D)
G = diag(random.normal(0, 1, D))
Pi = Pi_perm(D)
S = 12345    # do later

In [ ]:
V = dot(H, dot(G, dot(Pi, dot(H, B))))
norm_fact = sqrt((V**2).sum(1)[0])
Vn = V / norm_fact

In [ ]:
if True:
    print 'Row norms of V:', (V**2).sum(1)[:10]
    print 'Col norms of V:', (V**2).sum(0)[:10]
    print 'Row norms of Vn:', (Vn**2).sum(1)[:10]
    print 'Col norms of Vn:', (Vn**2).sum(0)[:10]

In [ ]:
def norm_angle_hist(mat, fc, rows=True):
    assert mat.shape[0] == mat.shape[1], 'square only'
    D = mat.shape[0]
    if rows:
        norms = sqrt((mat**2).sum(1))
        mat_norm = mat / norms.reshape((-1, 1))
        prods = dot(mat_norm, mat_norm.T)
    else:
        # Cols
        norms = sqrt((mat**2).sum(0))
        mat_norm = mat / norms.reshape((1, -1))
        prods = dot(mat_norm.T, mat_norm)
    angles = arccos(prods[~eye(D, dtype=bool)]) * 180 / pi
    subplot(2, 1, 1)
    hist(norms)
    subplot(2, 1, 2)
    hist(angles, bins=20, fc=fc, normed=True)
    title('mean %f, std %f' % (angles.mean(), angles.std()))

In [ ]:
h_prods = dot(Vn, Vn.T)

In [ ]:
norm_angle_hist(V, (0, 0, 1, .2))
norm_angle_hist(V.T ,(1, 0, 0, .2))

In [ ]:
bigG = random.normal(0, 1, (D, D))

In [ ]:
norm_angle_hist(bigG, (0, 0, 1, .2))
norm_angle_hist(bigG.T ,(1, 0, 0, .2))

How to compute normalization factor

In [ ]:
(V**2).sum(1)[:10]

In [ ]:
(G**2).sum()

In [ ]:
((H[0]**2).sum() * (G**2).sum())

In [ ]:
(D * (G**2).sum())

## Tensorflow, Complete Fastfood Transform

In [ ]:
in_x = tf.placeholder('float32', name='in_x')

Project from d to D!

Note: when D is a power of two, normalization is perfect. When it's not, the normalization is approximate. The approximation is better when D is large (D = 1000 seems to produce transformations with jacobian values within 1% of 1)

In [ ]:
d = 5
D = 1024

ff_vars, xform = tf_fastfood_transform(in_x, d, D)
sess.run(tf.global_variables_initializer())

x0 = np.zeros(d)
x1 = x0.copy(); x1[0] += 1
x2 = x0.copy(); x2[1] += 1
x3 = x0.copy(); x3[2] += 1

print 'Projecting from %d to %d' % (d, D)
print 'Norm offsets when changing the first three components in the reduced space:'
print ((sess.run(xform, feed_dict={in_x: x0}) - sess.run(xform, feed_dict={in_x: x1}))**2).sum()
print ((sess.run(xform, feed_dict={in_x: x0}) - sess.run(xform, feed_dict={in_x: x2}))**2).sum()
print ((sess.run(xform, feed_dict={in_x: x0}) - sess.run(xform, feed_dict={in_x: x3}))**2).sum()
